# X-Plane 11 Aircraft Taxi Controller

In this notebook, you will implement a controller for the aircraft taxi problem and observe it in action!

First, let's import some of the python packages we will need:

In [1]:
import time
import numpy as np

from xpc3 import *
from xpc3_helper import *

Next, we need to get X-Plane 11 ready to go. We will follow the same steps as we did in the X-Plane 11 tutorial notebook:

1. **Open X-Plane 11.** Look for the X-Plane 11 icon in the dock at the bottom of your screen (it should be towards the right). Click this icon and X-Plane 11 will open. A window may pop up saying that there is an update available. If this happens, you can just click "ignore".
2. From the main menu of X-Plane 11, click "New Flight". This should bring up a window with some flight configuration options.
3. In the AIRCRAFT section, select the Cessna Skyhawk.
4. In the location section, select Grant Co Intl (ID is KMWH).
5. You can leave the weather as clear.
6. In the TIME OF DAY section, select a time between 8AM and 10AM local.
7. Click the "Start Flight" button at the bottom right of the screen. It make take a minute or so to load the flight. Once it loads, you should see the front of the aircraft pointing down a runway. We are now ready to start controlling it!

Connect to X-Plane 11 by creating a client that we can use to interface with the simulator:

In [2]:
client = XPlaneConnect()

## Simulation Functions
To simulate our taxi controller, we need to implement a *closed-loop system*. This can be a big task, so let's break it down into smaller ones! At each time step, we want to:

1. Get the current state of the system
2. Determine our control input using the state (steering angle)
3. Feed the current state and control input into the dynamics function to determine the next state
4. Go back to step 1!

Let's make a separate function for steps 1-3. The function for step 1 returns the current state and has been done for you:

### 1. Get the current state
This one has been done for you and just uses the data from X-Plane 11 to return the current crosstrack and heading error.

In [3]:
def getState(client):
    """ Returns the true crosstrack error (meters) and
        heading error, (degrees) to simulate fully 
        oberservable control
        Args:
            client: XPlane Client
    """
    cte, _, he = xpc3_helper.getHomeState(client)
    return cte, he


### 2. Determine the control input

Remember all of that tuning you did on you controller? Let's test out those values here! Complete the function below with the best values you found for your controller:

In [5]:
def getProportionalControl(client, cte, he):
    """ Returns steering angle command using proportional control
        Args:
            client: XPlane Client
            cte: current estimate of the crosstrack error, x (meters)
            he: current estimate of the heading error, theta (degrees)
    """
    # STUDENT CODE START
    cte_multiplier = -0.74 # REPLACE THIS WITH YOUR VALUE!
    he_multiplier = -0.44 # REPLACE THIS WITH YOUR VALUE!
    # STUDENT CODE END
    return cte_multiplier * cte + he_multiplier * he

### 3. Get the next state
Fill in the dynamics function to return the next state. Remember the dynamics equations we found:
- he_next = he + phi
- cte_next = cte + v * sin (theta)
- dtp_next = dtp + v * cos (theta)

For sin and cos, you can use the `np.sin` and `np.cos` functions. NOTE: these functions expect the input angle to be in RADIANS. Just like degrees, radians are another unit of measurement 

In [ ]:
def dynamics(cte, dtp, theta, phi_deg, dt=0.05, v=5, L=5):
    """ Dubin's car dynamics model (returns next state)
        Args:
            x: current crosstrack error (meters)
            y: current downtrack position (meters)
            theta: current heading error (degrees)
            phi_deg: steering angle input (degrees)
            -------------------------------
            dt: time step (seconds)
            v: speed (m/s)
            L: distance between front and back wheels (meters)
    """
    
    theta_rad = np.deg2rad(theta)
    phi_rad = np.deg2rad(phi_deg)

    x_dot = v * np.sin(theta_rad)
    y_dot = v * np.cos(theta_rad)
    theta_dot = (v / L) * np.tan(phi_rad)

    x_prime = x + x_dot * dt
    y_prime = y + y_dot * dt
    theta_prime = theta + np.rad2deg(theta_dot) * dt

    return x_prime, theta_prime, y_prime

In [ ]:
def simulate_controller()